In [3]:
import pickle
import random

In [4]:
# random.seed(42)

In [5]:
with open("data/positive_labels.p", "rb") as f:
    positive_labels = pickle.load(f)

with open("data/positive_labels_per_artist.p", "rb") as f:
    positive_labels_per_artist = pickle.load(f)

with open("data/artist_dict.p", "rb") as f:
    artist_dict = pickle.load(f)

with open("data/artist_names.p", "rb") as f:
    artist_names = pickle.load(f)

In [6]:
from sklearn.model_selection import train_test_split
import torch

In [7]:
train, dev = train_test_split(positive_labels, train_size=0.8, random_state=42)
dev, test = train_test_split(dev, train_size=0.5, random_state=42)

In [8]:
def create_triples(name_pairs, all_names, artist_dict, positive_labels_per_artist):
    anchor_ids = []
    positive_ids = []
    negative_ids = []
    not_found = []
    for i, pair in enumerate(name_pairs):
        anchor_name = pair[0]
        positive_name = pair[1]
        if anchor_name in artist_dict and positive_name in artist_dict:
            anchor_id = artist_dict[anchor_name]
            positive_id = artist_dict[pair[1]]
            random.seed(i)
#             count = 0
            while True:
                sample = random.sample(all_names, 1)[0]
                if sample not in positive_labels_per_artist[anchor_name]:
                    negative_id = artist_dict[sample]
                    break
            anchor_ids.append(anchor_id)
            positive_ids.append(positive_id)
            negative_ids.append(negative_id)
#                     count += 1
        else:
            not_found.append(pair)
        
    return anchor_ids, positive_ids, negative_ids, not_found

In [9]:
# len(train), len(train_a_ids)

In [10]:
train_a_ids, train_p_ids, train_n_ids, train_not_found = create_triples(train, 
                                                                        artist_names, 
                                                                        artist_dict, 
                                                                  positive_labels_per_artist)


In [11]:
dev_a_ids, dev_p_ids, dev_n_ids, dev_not_found = create_triples(dev, 
                                                                artist_names, 
                                                                artist_dict, 
                                                                positive_labels_per_artist)


In [12]:
test_a_ids, test_p_ids, test_n_ids, test_not_found = create_triples(test, 
                                                                  artist_names, 
                                                                  artist_dict, 
                                                                  positive_labels_per_artist)


In [13]:
train_a_ids = torch.tensor(train_a_ids).unsqueeze(1)
train_p_ids = torch.tensor(train_p_ids).unsqueeze(1)
train_n_ids = torch.tensor(train_n_ids).unsqueeze(1)

dev_a_ids = torch.tensor(dev_a_ids).unsqueeze(1)
dev_p_ids = torch.tensor(dev_p_ids).unsqueeze(1)
dev_n_ids = torch.tensor(dev_n_ids).unsqueeze(1)

test_a_ids = torch.tensor(test_a_ids).unsqueeze(1)
test_p_ids = torch.tensor(test_p_ids).unsqueeze(1)
test_n_ids = torch.tensor(test_n_ids).unsqueeze(1)


with open("data/og_train.p", "wb") as f:
    data = {
        "a_ids" : train_a_ids,
        "p_ids" : train_p_ids,
        "n_ids" : train_n_ids
    }
    pickle.dump(data, f)

with open("data/og_dev.p", "wb") as f:
    data = {
        "a_ids" : dev_a_ids,
        "p_ids" : dev_p_ids,
        "n_ids" : dev_n_ids
    }
    pickle.dump(data, f)

with open("data/og_test.p", "wb") as f:
    data = {
        "a_ids" : test_a_ids,
        "p_ids" : test_p_ids,
        "n_ids" : test_n_ids
    }
    pickle.dump(data, f)